In [52]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [53]:
# Load and normalizde the data

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batchSize = 5
validSize = 0.2 # use 20% of train set as validation

trainValidSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainSet, validSet = torch.utils.data.random_split(trainValidSet, [int(len(trainValidSet)*(1-validSize)), int(len(trainValidSet)*validSize)])

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True)
validLoader = torch.utils.data.DataLoader(validSet, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [54]:
# Writer for tensorBoard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [55]:
len(trainLoader)*batchSize, len(validLoader)*batchSize, len(testLoader)*batchSize, next(iter(testLoader))[0][0].shape

(40000, 10000, 10000, torch.Size([3, 32, 32]))

In [56]:
# Define the network class

class ConvNet(nn.Module):
    def __init__(self, activation):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 10)
        self.activation = activation

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x


In [57]:
def trainNetwork(network, optimizer, lossFunction, epochs, writer):
    network.train()
    for epoch in range(epochs):
        
        ### TRAINING ###
        trainLoss = 0
        correctTrain = 0
        totalTrain = 0
        for batch_nr, (images, labels) in enumerate(trainLoader):

            # Predict
            predictions = network(images)

            # Get loss and backpropogate
            loss = lossFunction(predictions, labels)
            loss.backward() 

            # Optimize parameters (weights and biases) and remove gradients after
            optimizer.step() 
            optimizer.zero_grad()

            # Save loss for whole epoch
            trainLoss += loss.item()
            
            # Calculate training accuracy
            _, predictions = torch.max(predictions, 1) 
            correctTrain += (predictions == labels).sum().item() 
            totalTrain += len(images)
        

        trainLoss /= len(trainLoader)
        trainAccuracy = 100 * correctTrain / totalTrain
        writer.add_scalar(f"Loss/train - {network.activation.__name__}/{type(optimizer).__name__}", trainLoss, epoch)
        writer.add_scalar(f"Accuracy/train - {network.activation.__name__}/{type(optimizer).__name__}", trainAccuracy, epoch)

        ### VALIDATION ###
        validLoss = 0
        correctValid = 0
        totalValid = 0
        for batch_nr, (images, labels) in enumerate(validLoader):

            # Predict
            predictions = network(images)

            # Get loss
            loss = lossFunction(predictions, labels)

            # Save loss for whole epoch
            validLoss += loss.item()

            # Calculate vaildation accuracy
            _, predictions = torch.max(predictions, 1) 
            correctValid += (predictions == labels).sum().item() 
            totalValid += len(images)

        validLoss /= len(validLoader)
        validAccuracy = 100 * correctValid / totalValid
        writer.add_scalar(f"Loss/valid - {network.activation.__name__}/{type(optimizer).__name__}", validLoss, epoch)
        writer.add_scalar(f"Accuracy/train - {network.activation.__name__}/{type(optimizer).__name__}", validAccuracy, epoch)

        # Print reuslt of epoch
        print(f'Epoch [{epoch+1}/{epochs}] \t Training Loss: {trainLoss} \t Validation Loss: {validLoss} \t Traning Acc: {trainAccuracy}% \t Validation Acc: {validAccuracy}%')

    writer.flush()

In [58]:
def testNetwork(network):
    network.eval()
    correctTest = 0
    totalTest = 0

    ### TESTING ###
    with torch.no_grad(): 
        for batch_nr, (images, labels) in enumerate(testLoader):
            # Get predictions and get the amount of correct predicitons
            predictions = network(images)
            _, predictions = torch.max(predictions, 1) 
            correctTest += (predictions == labels).sum().item() 
            totalTest += len(images)
        
    print(f"Test Accuracy: {100 * correctTest / totalTest}%")

In [59]:
learningRate = 0.0001
epochs = 10

In [60]:
networkLeakyRelu = ConvNet(F.leaky_relu)

optimizerSGD = torch.optim.SGD(networkLeakyRelu.parameters(), lr=learningRate)
lossFunction = nn.CrossEntropyLoss()

In [61]:
trainNetwork(networkLeakyRelu, optimizerSGD, lossFunction, epochs, writer)

Epoch [1/10] 	 Training Loss: 2.3010705431997778 	 Validation Loss: 2.2988175995349884 	 Traning Acc: 12.0% 	 Validation Acc: 13.1%
Epoch [2/10] 	 Training Loss: 2.2951286836862566 	 Validation Loss: 2.2918897829055784 	 Traning Acc: 14.0275% 	 Validation Acc: 14.46%
Epoch [3/10] 	 Training Loss: 2.2854744682013988 	 Validation Loss: 2.2793265515565873 	 Traning Acc: 15.44% 	 Validation Acc: 14.86%
Epoch [4/10] 	 Training Loss: 2.265778324186802 	 Validation Loss: 2.2515283019542696 	 Traning Acc: 16.3725% 	 Validation Acc: 15.72%
Epoch [5/10] 	 Training Loss: 2.223016860470176 	 Validation Loss: 2.193535417318344 	 Traning Acc: 18.095% 	 Validation Acc: 19.18%
Epoch [6/10] 	 Training Loss: 2.1527261293977498 	 Validation Loss: 2.120692051231861 	 Traning Acc: 22.41% 	 Validation Acc: 22.87%
Epoch [7/10] 	 Training Loss: 2.091551159247756 	 Validation Loss: 2.0728317902088165 	 Traning Acc: 25.5275% 	 Validation Acc: 26.19%
Epoch [8/10] 	 Training Loss: 2.0524013607800007 	 Validation 

In [62]:
testNetwork(networkLeakyRelu)

Test Accuracy: 30.73%


In [64]:
networkLeakyRelu2 = ConvNet(F.leaky_relu)

optimizerAdam = torch.optim.Adam(networkLeakyRelu2.parameters(), lr=learningRate)
trainNetwork(networkLeakyRelu2, optimizerAdam, lossFunction, epochs, writer)

KeyboardInterrupt: 

In [ ]:
testNetwork(networkLeakyRelu)

Accuracy: 56.83%


In [ ]:
networkTanh = ConvNet(torch.tanh)
trainNetwork(networkTanh, optimizerAdam, lossFunction, epochs, writer)

Epoch [1/10] 	 Training Loss: 2.3051748233437537 	 Validation Loss: 2.30424448120594 	 Traning Acc: 9.5425% 	 Validation Acc: 9.86%
Epoch [2/10] 	 Training Loss: 2.305174823105335 	 Validation Loss: 2.3042444862127303 	 Traning Acc: 9.5425% 	 Validation Acc: 9.86%
Epoch [3/10] 	 Training Loss: 2.3051748223900796 	 Validation Loss: 2.3042444816827774 	 Traning Acc: 9.5425% 	 Validation Acc: 9.86%
Epoch [4/10] 	 Training Loss: 2.305174823194742 	 Validation Loss: 2.3042444821596146 	 Traning Acc: 9.5425% 	 Validation Acc: 9.86%
Epoch [5/10] 	 Training Loss: 2.305174824744463 	 Validation Loss: 2.304244488954544 	 Traning Acc: 9.5425% 	 Validation Acc: 9.86%
Epoch [6/10] 	 Training Loss: 2.3051748228669164 	 Validation Loss: 2.3042444829940796 	 Traning Acc: 9.5425% 	 Validation Acc: 9.86%
Epoch [7/10] 	 Training Loss: 2.3051748249530792 	 Validation Loss: 2.304244480252266 	 Traning Acc: 9.5425% 	 Validation Acc: 9.86%
Epoch [8/10] 	 Training Loss: 2.305174824178219 	 Validation Loss: 2.

In [ ]:
testNetwork(networkTanh)

Accuracy: 9.5%
